# Monte-Carlo Estimate of $\pi$

We want to estimate the number $\pi$ using a [Monte-Carlo method](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods) exploiting that the area of a quarter circle of unit radius is $\pi/4$ and that hence the probability of any randomly chosen point in a unit square to lie in a unit circle centerd at a corner of the unit square is $\pi/4$ as well.  So for N randomly chosen pairs $(x, y)$ with $x\in[0, 1)$ and $y\in[0, 1)$, we count the number $N_{circ}$ of pairs that also satisfy $(x^2 + y^2) < 1$ and estimage $\pi \approx 4 \cdot N_{circ} / N$.

[<img src="https://upload.wikimedia.org/wikipedia/commons/8/84/Pi_30K.gif" 
     width="50%" 
     align=top
     alt="PI monte-carlo estimate">](https://en.wikipedia.org/wiki/Pi#Monte_Carlo_methods)

## Core Lessons

- Resilience against dying workers

## Set up a Slurm cluster

In [1]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

/p/project/cecam/rath1/miniconda3_20190521/envs/dask_jobqueue_workshop/lib/python3.7/site-packages/docrep/__init__.py:341: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
  s = dedents('\n' + '\n'.join(lines[first:]))


In [2]:
import os

In [3]:
cluster = SLURMCluster(
    cores=24,
    processes=2,
    memory="100GB",
    shebang='#!/usr/bin/env bash',
    queue="batch",
    walltime="00:30:00",
    local_directory='/tmp',
    death_timeout="15s",
    interface="ib0",
    log_directory=f'{os.environ["SCRATCH_cecam"]}/{os.environ["USER"]}/dask_jobqueue_logs/',
    project="ecam",
    name="resilient_clusters")

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://10.80.32.41:33451 Dashboard: http://10.80.32.41:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## The job scripts

In [5]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J resilient_clusters
#SBATCH -e /p/scratch/cecam/rath1/dask_jobqueue_logs//resilient_clusters-%J.err
#SBATCH -o /p/scratch/cecam/rath1/dask_jobqueue_logs//resilient_clusters-%J.out
#SBATCH -p batch
#SBATCH -A ecam
#SBATCH -n 1
#SBATCH --cpus-per-task=24
#SBATCH --mem=94G
#SBATCH -t 00:30:00
JOB_ID=${SLURM_JOB_ID%;*}



/p/project/cecam/rath1/miniconda3_20190521/envs/dask_jobqueue_workshop/bin/python -m distributed.cli.dask_worker tcp://10.80.32.41:33451 --nthreads 12 --nprocs 2 --memory-limit 50.00GB --name resilient_clusters--${JOB_ID}-- --death-timeout 15s --local-directory /tmp --interface ib0



## Scale the cluster to two nodes

In [6]:
cluster.adapt(minimum=12, maximum=12)  # will lead to six jobs

## The Monte Carlo Method

In [7]:
import dask.array as da
import numpy as np

In [8]:
def calc_pi_mc(size_in_bytes, chunksize_in_bytes=200e6):
    """Calculate PI using a Monte Carlo estimate."""
    
    size = int(size_in_bytes / 8)
    chunksize = int(chunksize_in_bytes / 8)
    
    xy = da.random.uniform(0, 1,
                           size=(size / 2, 2),
                           chunks=(chunksize / 2, 2))
    
    in_circle = ((xy ** 2).sum(axis=-1) < 1)
    pi = 4 * in_circle.mean()

    return pi

In [9]:
def print_pi_stats(size, pi, time_delta, num_workers):
    """Print pi, calculate offset from true value, and print some stats."""
    print(f"{size / 1e9} GB\n"
          f"\tMC pi: {pi : 13.11f}"
          f"\tErr: {abs(pi - np.pi) : 10.3e}\n"
          f"\tWorkers: {num_workers}"
          f"\t\tTime: {time_delta : 7.3f}s")

## The actual calculations

We loop over different volumes of double-precision random numbers and estimate $\pi$ as described above.

In [10]:
from time import time

In [11]:
for size in (1e9 * n for n in (1, 10, 100)):
    
    start = time()
    pi = calc_pi_mc(size).compute()
    elaps = time() - start

    print_pi_stats(size, pi, time_delta=elaps,
                   num_workers=len(cluster.scheduler.workers))

1.0 GB
	MC pi:  3.14145440000	Err:  1.383e-04
	Workers: 6		Time:  14.832s
10.0 GB
	MC pi:  3.14162355200	Err:  3.090e-05
	Workers: 6		Time:   1.158s
100.0 GB
	MC pi:  3.14158572480	Err:  6.929e-06
	Workers: 12		Time:   4.409s


## What happens if a worker dies?

We'll find out all "our" job ids, mark a few of them non-preemptible, filter for the preemptible jobs, and define a function to kill one randomly selected preemptible job.

In [12]:
def get_current_jobs():
    current_jobs = !squeue | grep R | grep $USER | grep resil | awk '{print $1}'
    return current_jobs

In [13]:
non_preemptible_jobs = get_current_jobs()[:2]
non_preemptible_jobs

['7191395', '7191396']

In [14]:
def get_preemptible_jobs():
    return list(filter(lambda j: j not in non_preemptible_jobs, get_current_jobs()))

In [15]:
import random

def kill_random_preemptible_job():
    preemptible_jobs = get_preemptible_jobs()
    if preemptible_jobs:
        worker_to_kill = random.choice(preemptible_jobs)
        print(f"will cancel job {worker_to_kill}")
        !scancel {worker_to_kill}

## Let's start a computation with disappearing workers

In [16]:
pi = calc_pi_mc(1e12, 500e6)

In [17]:
pi = client.compute(pi)
pi

<Future: status: pending, key: finalize-2a7825eef9b2726c91452b1374ee6eca>

In [18]:
from time import sleep

In [19]:
sleep(5)

while not pi.done():
    kill_random_preemptible_job()
    sleep(10)

will cancel job 7191399
will cancel job 7191400
will cancel job 7191397


In [20]:
pi

<Future: status: error, key: finalize-2a7825eef9b2726c91452b1374ee6eca>

In [21]:
print(pi.result())

KilledWorker: ("('sum-sum-aggregate-uniform-mean_chunk-d15a005003e49b0f8054e3bbec8f3ce1', 1688)", 'tcp://10.80.35.39:40738')

## What happened?

The Dask scheduler keeps a suspiciousness counter for each task it manages.  Whenever a worker dies, all tasks that belong to the worker at the time of its death will have their suspiciousness increased by one. In doing so, the scheduler has no way of telling which exact task was responsible for the death of the worker and just flag all of them as bad.

All tasks with suspiciousness `>= 3` (default) are considered bad and won't be rescheduled.

## Make dask more resilient

We can increase the number of allowed failures.  Let's practically disable the threshold and re-do the calculation.

In [22]:
cluster.scheduler.allowed_failures = 1000

In [23]:
pi = calc_pi_mc(1e12, 500e6)

In [24]:
pi = client.compute(pi)
pi

<Future: status: pending, key: finalize-054a7564edc5c9907858eac3e79c0d6d>

In [26]:
sleep(5)

while not pi.done():
    kill_random_preemptible_job()
    sleep(10)

will cancel job 7191405
will cancel job 7191398
will cancel job 7191406
will cancel job 7191402
will cancel job 7191411


In [27]:
pi

<Future: status: finished, type: float64, key: finalize-054a7564edc5c9907858eac3e79c0d6d>

In [28]:
print(pi.result())

3.141595365824


## Summary

Dask very consciously handles failing tasks and can be made very resilient against suddenly disappearing ressources.

## Complete listing of software used here

In [29]:
%pip list

Package            Version          
------------------ -----------------
asciitree          0.3.3            
aspy.yaml          1.2.0            
backcall           0.1.0            
bokeh              1.1.0            
certifi            2019.3.9         
cfgv               1.6.0            
cftime             1.0.3.4          
Click              7.0              
cloudpickle        1.0.0            
cycler             0.10.0           
cytoolz            0.9.0.1          
dask               1.2.0            
dask-jobqueue      0.4.1+32.g9c3371d
decorator          4.4.0            
distributed        1.27.1           
docrep             0.2.5            
fasteners          0.14.1           
heapdict           1.0.0            
identify           1.4.3            
importlib-metadata 0.13             
ipykernel          5.1.1            
ipython            7.5.0            
ipython-genutils   0.2.0            
jedi               0.13.3           
Jinja2             2.10.1           
j

In [30]:
%conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://conda.anaconda.org/conda-forge/linux-64/git-lfs-2.7.2-0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ca-certificates-2019.3.9-hecc5488_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgcc-ng-8.2.0-hdf63c60_1.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libgfortran-ng-7.3.0-hdf63c60_0.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/libstdcxx-ng-8.2.0-hdf63c60_1.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/bzip2-1.0.6-h14c3975_1002.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/expat-2.2.5-hf484d3e_1002.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/icu-58.2-hf484d3e_1000.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/jpeg-9c-h14c3975_1001.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/libffi-3.2.1-he1b5a44_1006.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/